# PAGERANK COMPUTATION USING APACHE SPARK:

Using Apache Spark, PageRank can be implemented efficiently to process large-scale web graphs. Spark's in-memory processing and iterative computation capabilities make it well-suited for PageRank, which requires multiple iterations to compute stable rankings. Spark’s Resilient Distributed Datasets (RDDs) and its graph-processing library, GraphX, simplify the implementation by abstracting complex operations and managing distributed data seamlessly.


In [ ]:
!pip install --quiet pyspark

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("word count").setMaster("local")
sc = SparkContext(conf=conf)

In [ ]:
import time

In [ ]:
## spark_input.txt
# 1 31
# 1 12
# 1 82
# 1 42
# 1 58
# 2 18
# 2 94
# 2 15


# n = float(input("Enter Total Number of Node: "))
n = 100.0
print("Enter Total Number of Node: ",n)

Enter Total Number of Node:  100.0


In [ ]:
spark_input_lines = sc.textFile("/content/spark_input.txt")

## Mapping Values and generating Key-Value Pairs

In [ ]:
links = spark_input_lines.map(lambda s: tuple(s.split())).distinct().groupByKey().cache()

# initialize ranks
# (1, 0.01)
# (2, 0.01)
ranks = links.mapValues(lambda v: float(1/n))

## links::
# (1, [31, 12, 82, 42, 58])
# (2, [18, 94, 15])

## Execution Time Without Sorting:

In [ ]:
start_time = time.time()
iters = 30
for i in range(1, iters + 1):
        contribs = links.join(ranks).values().flatMap(lambda x: [(url, x[1] / len(x[0])) for url in x[0]])

        # NewRank = 0.15*(1/n) + 0.85*AggregatedRank
        # This balances random jumps(0.15) and rank propagation(0.85)
        ranks = contribs.reduceByKey(lambda x, y: x + y).mapValues(lambda rank: 0.15*(float(1/n)) + 0.85 * rank)

distributed_page_ranks = ranks.collect()
end_time = time.time()
print("Time taken = {:.2f} seconds ".format(end_time - start_time))

Time taken = 16.83 seconds 


In [ ]:
for rank in distributed_page_ranks:
    print(rank[0] + " has rank: " + str(rank[1]) + ".")

31 has rank: 0.007912909048990807.
12 has rank: 0.0029002702076193396.
82 has rank: 0.004300015762032965.
42 has rank: 0.0068519929455248315.
58 has rank: 0.002555752339031856.
18 has rank: 0.00853563895695363.
94 has rank: 0.011459293937537876.
15 has rank: 0.005903534626598428.
51 has rank: 0.0029102797201208015.
68 has rank: 0.014834997221698754.
41 has rank: 0.005657997641382228.
17 has rank: 0.0029542802386190153.
87 has rank: 0.004221202436274588.
52 has rank: 0.005810643895646598.
65 has rank: 0.004063834902536787.
47 has rank: 0.004790322252899333.
22 has rank: 0.004150935690092405.
59 has rank: 0.0062592520387759875.
53 has rank: 0.0045341970430504575.
90 has rank: 0.00561464759290067.
70 has rank: 0.00498891766662224.
11 has rank: 0.005117182597508121.
48 has rank: 0.004066676182030435.
9 has rank: 0.0059410335328304986.
56 has rank: 0.011666468124454463.
62 has rank: 0.005829629066034387.
30 has rank: 0.004923465529542245.
10 has rank: 0.009397590437605184.
74 has rank: 0.00

## Execution Time With Sorting:

In [ ]:
start_time = time.time()
iters = 30
for i in range(1, iters + 1):
        contribs = links.join(ranks).values().flatMap(lambda x: [(url, x[1] / len(x[0])) for url in x[0]])

        # NewRank = 0.15*(1/n) + 0.85*AggregatedRank
        # This balances random jumps(0.15) and rank propagation(0.85)
        ranks = contribs.reduceByKey(lambda x, y: x + y).mapValues(lambda rank: 0.15*(float(1/n)) + 0.85 * rank)

distributed_sorted_page_ranks = sorted(ranks.collect(), key=lambda x: x[1], reverse=True)
end_time = time.time()
print("Time taken = {:.2f} seconds ".format(end_time - start_time))

Time taken = 15.10 seconds 


In [ ]:
for rank in distributed_sorted_page_ranks:
    print(rank[0] + " has rank: " + str(rank[1]) + ".")

21 has rank: 0.01857001067867647.
68 has rank: 0.014834997168476199.
29 has rank: 0.012651967304744181.
84 has rank: 0.01186658503716918.
56 has rank: 0.011666468082602013.
94 has rank: 0.011459293892284282.
85 has rank: 0.010670528389787855.
75 has rank: 0.010511580907549844.
96 has rank: 0.010191068078801001.
28 has rank: 0.009923275820605369.
7 has rank: 0.009690563328640222.
10 has rank: 0.00939759040767489.
8 has rank: 0.009051510713942881.
49 has rank: 0.008565875702038187.
18 has rank: 0.008535638926668225.
34 has rank: 0.008078084823032745.
31 has rank: 0.007912909023144045.
24 has rank: 0.007772831138870609.
73 has rank: 0.007384687041864331.
83 has rank: 0.007126640764617028.
38 has rank: 0.0071222088743474275.
42 has rank: 0.006851992924282962.
97 has rank: 0.006833379561105609.
54 has rank: 0.006715841976995648.
86 has rank: 0.006391712874028642.
14 has rank: 0.006354453267559302.
59 has rank: 0.006259252022483876.
32 has rank: 0.006192809878785406.
71 has rank: 0.006166305